# Guided Investigation - Anomaly Lookup

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 - AzureML<br>
__Required Packages:__ Azure-Sentinel-Utilities<br>
__Platforms Supported:__  Azure Machine Learning Notebooks
     
__Data Source Required:__ Log Analytics tables 
    
### Description
Gain insights into the possible root cause of an alert by searching for related anomalies on the corresponding entities around the alert’s time. This notebook will provide valuable leads for an alert’s investigation, listing all suspicious increase in event counts or their properties around the time of the alert, and linking to the corresponding raw records in Log Analytics for the investigator to focus on and interpret.

<font>You may need to select Python 3.6 - AzureML on Azure Machine Learning Notebooks.</font>

## Table of Contents

1. Initialize Azure Resource Management Clients
2. Looking up for anomaly entities

## 1. Initialize Azure Resource Management Clients

In [ ]:
# only run once
!pip install --upgrade Azure-Sentinel-Utilities

In [ ]:
# Set variables from config.json
from SentinelWidgets import WidgetViewHelper
from SentinelUtils import ConfigReader
tenant_id, subscription_id, resource_group, workspace_id, workspace_name  = ConfigReader.read_config_values('config.json');

In [ ]:
from azure.loganalytics import LogAnalyticsDataClient
from azure.loganalytics.models import QueryBody
from azure.mgmt.loganalytics import LogAnalyticsManagementClient
import SentinelAzure
from SentinelAnomalyLookup import AnomalyFinder, AnomalyLookupViewHelper

from pandas.io.json import json_normalize
import sys
import timeit
import datetime as dt
import pandas as pd
import copy
from IPython.display import HTML

In [ ]:
# Authentication to Log Analytics
from azure.common.client_factory import get_client_from_cli_profile
from azure.common.credentials import get_azure_cli_credentials
!az login --tenant $tenant_id
la_client = get_client_from_cli_profile(LogAnalyticsManagementClient, subscription_id = subscription_id)
la = SentinelAzure.azure_loganalytics_helper.LogAnalyticsHelper(la_client)
creds, _ = get_azure_cli_credentials(resource="https://api.loganalytics.io")
la_data_client = LogAnalyticsDataClient(creds)

## 2. Looking up for anomaly entities

In [ ]:
import ipywidgets as widgets
#DateTime format: 2019-07-15T07:05:20.000
q_timestamp = widgets.Text(value='2020-07-08',description='DateTime: ')
display(q_timestamp)
#Entity format: computer
q_entity = widgets.Text(value='computer',description='Entity: ')
display(q_entity)

In [ ]:
anomaly_lookup = AnomalyFinder(workspace_id, la_data_client)
selected_tables = WidgetViewHelper.select_multiple_tables(anomaly_lookup)
display(selected_tables)

In [ ]:
# This action may take a few minutes or more, please be patient.
start = timeit.default_timer()
anomalies, queries = anomaly_lookup.run(q_timestamp.value, q_entity.value, list(selected_tables.value))

if anomalies is not None and len(anomalies) > 0:
    display(anomalies)

print('======= Task completed ===========')
print('Elapsed time: ', timeit.default_timer() - start, ' seconds')

if queries is not None:
    url = WidgetViewHelper.construct_url_for_log_analytics_logs(tenant_id, subscription_id, resource_group, workspace_name)
    print('======= Clicking the URL to go to Log Analytics =======')
    print(url)
    WidgetViewHelper.display_html(WidgetViewHelper.copy_to_clipboard(url, queries, 'Add queries to clipboard then paste to Logs'))
else:
    print('0 records found.')